In [64]:
import pandas as pd
import numpy as np
import re
from itertools import accumulate

In [153]:
emails = pd.read_csv(r'C:\Users\amrenkumar\Desktop\TestData\full-output1.csv')

In [154]:
emails.shape

(27226, 17)

In [155]:
emails_clean = emails[~emails.interaction_content.isnull()]

In [156]:
emails_clean = emails_clean.interaction_content

In [157]:
emails_clean.drop_duplicates(keep='first', inplace=True)

C:\Anaconda\lib\site-packages\pandas\core\base.py:1241: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return self._update_inplace(result)


In [158]:
emails_clean.head()

4     EBS Direct - Liquidity Provider - Volumes and ...
5                      5058239 26 NHIK7E6KLVR5 2345 3 0
6     WMR Intra Day Spot Rates,, DATE, 01/01/2015 23...
9     WMR Intra Day Spot Rates,, DATE, 02/01/2015 00...
14    WMR Intra Day Spot Rates,, DATE, 02/01/2015 01...
Name: interaction_content, dtype: object

In [159]:
email_clean= emails_clean.str.replace("[^a-zA-Z0-9. ]", "")
#email_clean= emails_clean.interaction_content.str

In [160]:
for i in email_clean[0:1]:
    print(i)

EBS Direct  Liquidity Provider  Volumes and Rankings  Prev Month 140 Page 1 of 1 C Patten01Jan201564038 PM v 6.0 Start Date 01Dec2014 End Date 31Dec2014 Bank of America  Total EBS Direct Volume Ranking Rank Group Total Direct Volume US m Total Direct ADV  mday  of Total Direct Volume Americas Asia amp Oceania EMEA Japan MG1LP 51455 2237 16  20  6  57  16  MG2Bank of America 46512 2022 15  26  7  63  4  MG3LP 38133 1658 12  21  21  30  28  MG4LP 31955 1389 10  19  16  43  21  MG5LP 29879 1299 10  30  12  49  8  MG6LP 24122 1049 8  25  22  35  18  MG7LP 13728 597 4  30  18  42  10  MG8LP 13646 593 4  35  22  37  6  MG9LP 13616 592 4  9  23  38  29  MG10LP 13350 580 4  25  18  56  0  MG11LP 10258 446 3  16  31  23  30  MG12LP 10125 440 3  30  19  48  3  MG13LP 7115 309 2  30  16  51  3  MG14LP 5270 229 2  15  33  32  20  MG15LP 2943 128 1  57  43  MG16LP 928 400  9  91  MG17LP 580 250  78  22  MG18LP 474 210  100  Total 314090 13656 100  23  16  46  15  Customer  Volume Split  ICAP plc201

In [161]:
import nltk
from nltk.tag import pos_tag
from nltk import ne_chunk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\amrenkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\amrenkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\amrenkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amrenkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [162]:
sent_tokens=[]
for i in list(email_clean):
    sent_tokens.append(nltk.sent_tokenize(i))

In [163]:
sents = []
for sent in range(len(sent_tokens)):
    for i in range(len(sent_tokens[sent])):
        sents.append(sent_tokens[sent][i])
    

In [167]:
len(sents)

37426

In [165]:
sents[3]

'ICAP does not accept any liability for any loss or damage howsoever caused arising from any errors omissions or reliance on any information contained in this material.'

In [166]:
sents= list(set(sents))

In [168]:
def tokenizer(sent):
    tokens = nltk.word_tokenize(sent)
    tags = nltk.pos_tag(tokens)
    
    tag_p = []
    for i in range(len(tags)):
        tag_p.append(tags[i][1])
    
    #token_len = []
    #for i in tokens:
    #    token_len.append(len(i)+1)
        
    #list2 = list(accumulate(token_len))
    #list2 = [x -1 for x in list2 ]
    
    #list0 = [0]
    #for i in list2:
     #   list0.append(i+1)
        
    #data = pd.DataFrame(list(zip(tokens,list0,list2, tag_p)))
    data = pd.DataFrame(list(zip(tokens,tag_p)))
    return data

In [169]:
df_final= pd.DataFrame()
for i,sent in enumerate(sents):
    df = tokenizer(sent)
    df['Sent'] = "Sentence:" + str(i)
    df_final = df_final.append(df)

In [170]:
df_final.to_csv('Annotation.csv')

In [171]:
df_final.columns = ['Token','POS','Sent']

In [172]:
df_final = df_final[['Sent','Token','POS']]

In [173]:
from nltk.util import ngrams

In [174]:
grams_list=[]
for sent in sents:
    for i in range(5):
        grams_list.extend(list(ngrams(nltk.word_tokenize(sent),i+1)))

In [175]:
l1=[]
for i in grams_list:
    l1.append(' '.join(i))

In [176]:
company = pd.read_csv('company.csv')

In [177]:
company = company.TYPE.map(lambda x: x.lower())

In [178]:
company[2]

'01 communique laboratory incorporated'

In [179]:
len(l1)

24095811

In [180]:
time = pd.read_csv('time.csv')

In [181]:
time = time.Time.map(lambda x: x.lower())

In [182]:
email_tokens_multi_size = [x.lower() for x in l1]

In [183]:
email_tokens_single = [x.lower() for x in l1 if len(x.split())==1]

In [184]:
company_matched = frozenset(company).intersection(email_tokens_multi_size)

In [185]:
len(company_matched)

1050

In [186]:
time_matched = frozenset(time).intersection(email_tokens_multi_size)

In [187]:
currency = pd.read_csv(r'C:\Users\amrenkumar\Desktop\TestData\Currency.csv')

In [188]:
currency = currency.Currency.map(lambda x: x.lower())

In [189]:
currency_matched = frozenset(currency).intersection(email_tokens_multi_size)

In [190]:
len(currency_matched)

241

In [191]:
location = pd.read_csv(r'C:\Users\amrenkumar\Desktop\TestData\location.csv')

In [192]:
location = location.Location.map(lambda x: x.lower())

In [193]:
location_matched = frozenset(location).intersection(email_tokens_multi_size)

In [194]:
len(location_matched)

1608

In [195]:
def token_annotate(tokens,entity,entity_type):
    tokens_tagged = tokens
    if entity_type == 'company':
        b_entity = ' B-org'
        i_entity = ' I-org'
    elif entity_type == 'time':
        b_entity = ' B-tim'
        i_entity = ' I-tim'    
    elif entity_type == 'currency':
        b_entity = ' B-cur'
        i_entity = ' I-cur'               
    elif entity_type == 'location':
        b_entity = ' B-geo'
        i_entity = ' I-geo'               
        
    for i, v in enumerate(tokens):
        for i1, v1 in enumerate(entity):
            if len(v1.split())==5:
                if v == v1.split()[0] and v1.split()[1] == tokens[i+1] and v1.split()[2] == tokens[i+2] and v1.split()[3] == tokens[i+3] and v1.split()[4] == tokens[i+4]:
                    tokens_tagged[i]=tokens[i] + b_entity
                    tokens_tagged[i+1]=tokens[i+1] + i_entity
                    tokens_tagged[i+2]=tokens[i+2] + i_entity
                    tokens_tagged[i+3]=tokens[i+3] + i_entity
                    tokens_tagged[i+4]=tokens[i+4] + i_entity                    
            if len(v1.split())==4:
                if v == v1.split()[0] and v1.split()[1] == tokens[i+1] and v1.split()[2] == tokens[i+2] and v1.split()[3] == tokens[i+3]:
                    tokens_tagged[i]=tokens[i] + b_entity
                    tokens_tagged[i+1]=tokens[i+1] + i_entity
                    tokens_tagged[i+2]=tokens[i+2] + i_entity
                    tokens_tagged[i+3]=tokens[i+3] + i_entity
            if len(v1.split())==3:
                if v == v1.split()[0] and v1.split()[1] == tokens[i+1] and v1.split()[2] == tokens[i+2]:
                    tokens_tagged[i]=tokens[i] + b_entity
                    tokens_tagged[i+1]=tokens[i+1] + i_entity
                    tokens_tagged[i+2]=tokens[i+2] + i_entity
            if len(v1.split())==2:
                if v == v1.split()[0] and v1.split()[1] == tokens[i+1]:
                    tokens_tagged[i]=tokens[i] + b_entity
                    tokens_tagged[i+1]=tokens[i+1] + i_entity
            if len(v1.split())==1:
                if v == v1.split()[0]:
                    tokens_tagged[i]=tokens[i] + b_entity

                        
    return tokens_tagged

                

In [196]:
print(pd.datetime.now())

2019-10-23 18:55:56.721486


In [197]:
t = token_annotate(email_tokens_single, time_matched,'time')
print(pd.datetime.now())

2019-10-23 19:01:58.016589


In [198]:
len(email_tokens_single)

4893907

In [199]:
t_company = token_annotate(t, company_matched, 'company')
print(pd.datetime.now())

2019-10-23 23:44:08.676455


In [200]:
t_currency = token_annotate(t_company, currency_matched, 'currency')

In [202]:
t_location = token_annotate(t_currency, location_matched, 'location')

In [203]:
t1=[]
for i in t_location:
    t1.append(i.split())

In [119]:
t1

[['ebs'],
 ['direct'],
 ['liquidity'],
 ['provider'],
 ['volumes'],
 ['and'],
 ['rankings'],
 ['prev'],
 ['month'],
 ['140'],
 ['page', 'B-geo'],
 ['1'],
 ['of'],
 ['1'],
 ['c'],
 ['patten01jan201564038'],
 ['pm'],
 ['v'],
 ['6.0'],
 ['start'],
 ['date'],
 ['01dec2014'],
 ['end'],
 ['date'],
 ['31dec2014'],
 ['bank', 'B-org'],
 ['of', 'I-org'],
 ['america', 'I-org'],
 ['total'],
 ['ebs'],
 ['direct'],
 ['volume'],
 ['ranking'],
 ['rank', 'B-org'],
 ['group', 'I-org'],
 ['total'],
 ['direct'],
 ['volume'],
 ['us'],
 ['m'],
 ['total'],
 ['direct'],
 ['adv'],
 ['mday'],
 ['of'],
 ['total'],
 ['direct'],
 ['volume'],
 ['americas'],
 ['asia'],
 ['amp'],
 ['oceania'],
 ['emea'],
 ['japan', 'B-geo'],
 ['mg1lp'],
 ['51455'],
 ['2237'],
 ['16'],
 ['20'],
 ['6'],
 ['57'],
 ['16'],
 ['mg2bank'],
 ['of'],
 ['america'],
 ['46512'],
 ['2022'],
 ['15'],
 ['26'],
 ['7'],
 ['63'],
 ['4'],
 ['mg3lp'],
 ['38133'],
 ['1658'],
 ['12'],
 ['21'],
 ['21'],
 ['30'],
 ['28'],
 ['mg4lp'],
 ['31955'],
 ['1389'],


In [204]:
tagged_Tokens = pd.DataFrame(t1, columns=['Token','Tag', 'Tag1'])

In [205]:
tagged_Tokens.drop('Tag1', axis=1, inplace=True)

In [206]:
tagged_Tokens.Tag = tagged_Tokens.Tag.fillna('O')

In [207]:
df_annotated_final = pd.concat([df_final.reset_index(drop=True), 
                     tagged_Tokens["Tag"].reset_index(drop=True)], axis=1)

In [208]:
df_annotated_final.loc[(df_annotated_final.Tag=='B-geo') & (df_annotated_final.Token=='strong') , ['Tag']] = 'O'

In [209]:
df_annotated_final.to_csv('df_annotated.csv')

In [126]:
df_annotated_final.head()

,Sent,Token,POS,Tag
0,Sentence:0,EBS,NNP,O
1,Sentence:0,Direct,NNP,O
2,Sentence:0,Liquidity,NNP,O
3,Sentence:0,Provider,NNP,O
4,Sentence:0,Volumes,NNP,O


In [210]:
df_annotated_final.shape

(4893907, 4)

In [211]:
df_annotated_final[df_annotated_final.Tag!='O'].shape

(474048, 4)

In [130]:
sents[10]

'All information contained herein is for informational purposes only and is the intellectual property of ICAP.'

In [131]:
df_annotated_final = pd.read_csv('df_annotated.csv')

In [222]:
set(df_annotated_final.Tag)

{'B-cur', 'B-geo', 'B-org', 'B-tim', 'I-cur', 'I-geo', 'I-org', 'O'}

In [135]:
df_annotated_final.loc[(df_annotated_final.Tag=='B-cur') , ['Tag']] = 'O'

In [137]:
df_annotated_final.to_csv('df_annotated.csv')

In [212]:
sents_with_tags = list(set(df_annotated_final[df_annotated_final.Tag!='O'].Sent))

In [213]:
len(sents_with_tags)

21558

In [223]:
set(df_annotated_final_clean.Tag)

{'B-cur', 'B-geo', 'B-org', 'B-tim', 'I-cur', 'I-geo', 'I-org', 'O'}

In [215]:
df_annotated_final_clean = df_annotated_final[df_annotated_final['Sent'].isin(sents_with_tags)]

In [216]:
df_annotated_final_clean.shape

(4546348, 4)

In [217]:
df_annotated_final.shape

(4893907, 4)

In [221]:
df_annotated_final_clean.to_csv('df_annotated.csv')

In [150]:
df_annotated_final_clean.loc[(df_annotated_final_clean.Tag=='I-org') , ['Tag']] = 'I-comp'

In [224]:
df_annotated_final_clean

,Sent,Token,POS,Tag
0,Sentence:0,E,RB,O
1,Sentence:0,11,CD,O
2,Sentence:0,Sep19,NNP,O
3,Sentence:0,C,NNP,O
4,Sentence:0,132.20,CD,O
5,Sentence:0,S,NNP,O
6,Sentence:0,160.00,CD,O
7,Sentence:0,Brazil,NNP,B-geo
8,Sentence:0,receive,JJ,O
9,Sentence:0,Jan16,NNP,O
